# Pandas_2

## Groupby

SQL groupby 명령어와 같음  
split, apply, combine 과정의 연산을 함 
index가 같은 것 끼리 묶는 것 : split  
함수를 적용하는것 : apply  
적용한 함수의 결과를 모아 결과를 보여주는 것 : combine  

In [2]:
from pandas import Series, DataFrame
import pandas as pd 
import numpy as np

In [12]:
raw_data = {
    "Points" : [876,789,863,673,741,812,756,788,694,701,804,690],
    "Rank" : [1,2,2,3,3,4,1,1,2,4,1,2],
    "Team" : ["Rider","Rider","Devils","Devils","Kings","Kings","Kings","Kings","Riders","Royals","Royals","Riders"],
    "Year" : [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017]
}

df = DataFrame(raw_data,columns = ["Points","Rank","Team","Year"])
df

,Points,Rank,Team,Year
0,876,1,Rider,2014
1,789,2,Rider,2015
2,863,2,Devils,2014
3,673,3,Devils,2015
4,741,3,Kings,2014
5,812,4,Kings,2015
6,756,1,Kings,2016
7,788,1,Kings,2017
8,694,2,Riders,2016
9,701,4,Royals,2014


In [13]:
# df.groupby("묶음의 기준이 되는 컬럼")["연산 적용을 받는 컬럼"].적용함수()

#팀별로 묶은 뒤 포인트를 합산한 결과를 보여준다. 
df.groupby("Team")["Points"].sum()

Team
Devils    1536
Kings     3097
Rider     1665
Riders    1384
Royals    1505
Name: Points, dtype: int64

In [14]:
df.groupby("Team")["Points"].mean()

Team
Devils    768.00
Kings     774.25
Rider     832.50
Riders    692.00
Royals    752.50
Name: Points, dtype: float64

## Groupby - split 

groupby에 의해 split된 상태를 추출하는 것도 가능하다. 

In [36]:
#team을 기준으로 split
grouped = df.groupby("Team")

#
for name,group in grouped : 
    print(name)
    print(group)

Devils
   Points  Rank    Team  Year
2     863     2  Devils  2014
3     673     3  Devils  2015
Kings
   Points  Rank   Team  Year
4     741     3  Kings  2014
5     812     4  Kings  2015
6     756     1  Kings  2016
7     788     1  Kings  2017
Rider
   Points  Rank   Team  Year
0     876     1  Rider  2014
1     789     2  Rider  2015
Riders
    Points  Rank    Team  Year
8      694     2  Riders  2016
11     690     2  Riders  2017
Royals
    Points  Rank    Team  Year
9      701     4  Royals  2014
10     804     1  Royals  2015


In [35]:
print(type(grouped))
print(type(group))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
<class 'pandas.core.frame.DataFrame'>


## Groupby - apply 

추출된 group 정보에 세 가지 유형의 apply가 가능하다   

    1. aggregation : 요약된 통계정보를 추출   
    2. trnasformation : 해당 정보를 변환   
    3. filtration : 특정 정보를 제거, 보여주는 필터링  

In [38]:
#aggregation 
grouped.agg(sum)

,Points,Rank,Year
Team,,,
Devils,1536,5,4029
Kings,3097,9,8062
Rider,1665,3,4029
Riders,1384,4,4033
Royals,1505,5,4029


In [40]:
grouped.agg([np.sum,np.mean])

Points         Rank        Year        
          sum    mean  sum  mean   sum    mean
Team                                          
Devils   1536  768.00    5  2.50  4029  2014.5
Kings    3097  774.25    9  2.25  8062  2015.5
Rider    1665  832.50    3  1.50  4029  2014.5
Riders   1384  692.00    4  2.00  4033  2016.5
Royals   1505  752.50    5  2.50  4029  2014.5

In [41]:
#transformation : Group별로 연산을 처리해준다. 
score = lambda x : (x.max())
grouped.transform(score)

,Points,Rank,Year
0,876,2,2015
1,876,2,2015
2,863,3,2015
3,863,3,2015
4,812,4,2017
5,812,4,2017
6,812,4,2017
7,812,4,2017
8,694,2,2017
9,804,4,2015


In [42]:
## filter 
df.groupby("Team").filter(lambda x : len(x) >= 3)

,Points,Rank,Team,Year
4,741,3,Kings,2014
5,812,4,Kings,2015
6,756,1,Kings,2016
7,788,1,Kings,2017


## Hierarchical index 

여러개의 인덱스를 사용하는 것 

In [26]:
#groupby 사용시 묶음의 기준을 여러개 사용하는 것도 가능하다. 

#Team 별로 묶고 그 다음 year별로 묶어서 points를 더한 결과를 보여준다. 
#Team이 index 1이 되고 year가 index 2가 된다. 
h_index = df.groupby(["Team","Year"])["Points"].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Rider   2014    876
        2015    789
Riders  2016    694
        2017    690
Royals  2014    701
        2015    804
Name: Points, dtype: int64

In [20]:
# unstack : data를 matrix형태로 묶어준다. 
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,812.0,756.0,788.0
Rider,876.0,789.0,NaN,NaN
Riders,NaN,NaN,694.0,690.0
Royals,701.0,804.0,NaN,NaN


In [21]:
# reset_index() : index를 없애서 풀어준다. 
h_index.reset_index()

,Team,Year,Points
0,Devils,2014,863
1,Devils,2015,673
2,Kings,2014,741
3,Kings,2015,812
4,Kings,2016,756
5,Kings,2017,788
6,Rider,2014,876
7,Rider,2015,789
8,Riders,2016,694
9,Riders,2017,690


In [27]:
# swaplevel : index의 순서를 바꿔줌 
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2015  Kings     812
2016  Kings     756
2017  Kings     788
2014  Rider     876
2015  Rider     789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
Name: Points, dtype: int64

In [29]:
# sort_index(level=) : index를 정렬해주는 것 
h_index.sort_index(level = 0)

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Rider   2014    876
        2015    789
Riders  2016    694
        2017    690
Royals  2014    701
        2015    804
Name: Points, dtype: int64